In [107]:
# !git add .
# !git commit -m "Initial commit"
# !git pull
# !git push -u origin main

[main e4229ef] Initial commit
 3 files changed, 676 insertions(+), 67 deletions(-)
Already up to date.
branch 'main' set up to track 'origin/main'.


To github.com:thernandez7/Sports_Misinformation_Tool.git
   9601ed3..e4229ef  main -> main


## Sports Misinformation Classification Tool

### Connect to the Database

In [67]:
import mysql.connector
from sqlalchemy import create_engine
import pandas as pd

username = 'root'  
password = 'password'
host = 'localhost' 
database = 'sports_news_db'

connection = mysql.connector.connect(
    host=host,
    user=username,
    password=password,
    database=database
)

if connection.is_connected():
    print("Connected to the database successfully!")

engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}/{database}", echo=False)

Connected to the database successfully!


### Use this format to insert into the database

INSERT INTO articles (team_or_player, source, publication_date, content, trust_score, classification, link)
VALUES
('New York Yankees, Los Angeles Lakers', 'ESPN', '2024-10-27', 'Yankees article content example.', 85.00, 'real', 'https://example.com/article1'),
('Los Angeles Lakers', 'Twitter', '2024-10-27', 'Lakers article content example.', 60.00, 'fake', 'https://example.com/article2');


#### Table fields 
Table Name: articles

Fields: 

     id INT AUTO_INCREMENT PRIMARY KEY,
     
     team_or_player VARCHAR(200), (This can be a string of comma-separated values)
     
     source VARCHAR(200),
     
     publication_date DATE,
     
     content TEXT,
     
     trust_score DECIMAL(5, 2), 
     
     classification ENUM('real', 'fake', 'unknown') DEFAULT 'unknown',

     link VARCHAR(255)

### Simulate Tool Working

In [ ]:
team_or_player = input("Enter the team or player's name: ")

query = f"SELECT * FROM articles WHERE FIND_IN_SET('{team_or_player}', team_or_player) > 0"
df_result = pd.read_sql(query, con=engine, params={'team_or_player': team_or_player})

#get results
if not df_result.empty:
    print(f"Articles related to {team_or_player}:")
    display(df_result)
else:
    print(f"No articles found for {team_or_player}.")

### Get Article Entries from RSS Feeds-- \*this is not continuous*

In [95]:
import feedparser

#add urls to this list to parse
url_list = [
        "https://api.foxsports.com/v2/content/optimized-rss?partnerKey=MB0Wehpmuj2lUhuRhQaafhBjAJqaPU244mlTDK1i&size=30&tags=fs/nfl",#fox NFL
        "https://api.foxsports.com/v2/content/optimized-rss?partnerKey=MB0Wehpmuj2lUhuRhQaafhBjAJqaPU244mlTDK1i&size=30", #fox MLB
        "https://api.foxsports.com/v2/content/optimized-rss?partnerKey=MB0Wehpmuj2lUhuRhQaafhBjAJqaPU244mlTDK1i&size=30&tags=fs/nba", #fox NBA
        "https://www.sportscollectorsdaily.com/feed/", #Sports Collectors Daily 
        "https://www.espn.com/espn/rss/news", #ESPN top headlines
        "https://deadspin.com/rss/", #Deadspin 
        "https://news.sportslogos.net/feed/", #SportsLogos.Net
    
]
    
entries = [] #list of dictionaries
for url in url_list:
    feed = feedparser.parse(url)

    feed_title= feed.feed.title
    for entry in feed.entries:
        entry_title= entry.title
        entry_link= entry.link
        entry_published_date= entry.published
        entry_summary= entry.summary

        entries.append({
            "feed_title": feed_title,
            "entry_title": entry_title,
            "entry_link": entry_link,
            "entry_published_date": entry_published_date,
            "entry_summary": entry_summary,
        })

df = pd.DataFrame(entries)
print(df)

df.to_csv('RSS_sports_feeds.csv', index=False)

               feed_title                                        entry_title  \
0      FOX Sports Digital  NFL top-10 rankings: Chiefs top Lions; Steeler...   
1      FOX Sports Digital  2024 NFL odds Week 11: Lines, spreads for all ...   
2      FOX Sports Digital  Jared Goff does the unthinkable, plus 10 more ...   
3      FOX Sports Digital  Kyler Murray squarely in MVP race, and what el...   
4      FOX Sports Digital  Brock Purdy, 49ers SURVIVE In Christian McCaff...   
..                    ...                                                ...   
381  SportsLogos.Net News  NFL Week 9 Uniform Schedule: Giants, Eagles, P...   
382  SportsLogos.Net News  QMJHL’s Quebec Remparts Pay Homage to Beloved ...   
383  SportsLogos.Net News  Oregon Ducks Unveil Final “Generation O” Unifo...   
384  SportsLogos.Net News  Illinois Fighting Illini Unveil Military Appre...   
385  SportsLogos.Net News             Dodgers Set World Series Sales Records   

                                       

In [51]:
#if we want to get contents of whole article we would need to scrape it 
#use newspaper3k?





### Format DF to match DB table

In [105]:
from datetime import datetime
from dateutil import parser

df['publication_date'] = df['entry_published_date'].apply(lambda x: parser.parse(x).date()) #parse different date formats to date object format
dbdf = pd.read_csv('RSS_sports_feeds.csv')
dbdf['team_or_player'] = df['entry_title']
dbdf['source'] = df['feed_title']
dbdf['publication_date'] = df['publication_date'] 
dbdf['content'] = df['entry_summary']
dbdf['trust_score'] = 0.00  
dbdf['classification'] = 'unknown' 
dbdf['link'] = df['entry_link']

sports_DB_df = dbdf[['team_or_player', 'source', 'publication_date', 'content', 'trust_score', 'classification', 'link']]

#save to a new CSV 
sports_DB_df.to_csv('formatted_sports_posts_for_DB.csv', index=False)
print("done")

done


C:\Users\tizia\Anaconda4\Lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


## Define labeling approach to get classification and trust score to update sports_DB_df